In [1]:

#coding=utf-8
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import argparse
import numpy as np 
from keras import *
from keras.models import Sequential  
from keras.layers import *
from keras.utils.np_utils import to_categorical  
from keras.preprocessing.image import img_to_array  
from keras.callbacks import ModelCheckpoint  
from Models.utils import MaxUnpooling2D,MaxPoolingWithArgmax2D
from sklearn.preprocessing import LabelEncoder  
from PIL import Image  
import matplotlib.pyplot as plt  
import cv2
import random
import os
from tqdm import tqdm  
from keras import backend as K 
K.set_image_dim_ordering('th')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import time
import gdal
seed = 7  
np.random.seed(seed)  

Using TensorFlow backend.


In [2]:
# data for training  
def generateData(batch_size,img_w,img_h,n_label,image_names=[],label_names=[]): 
    print ('gen-Sub-Image-Data...')
    image_filepath ='D:\Python\seg-data\data_MB/'
    batch_num=0
    while True:   
        bs=batch_size
        
        dataset = gdal.Open(image_filepath+image_names[batch_num%len(image_names)])
        im_width = dataset.RasterXSize #栅格矩阵的列数
        im_height = dataset.RasterYSize #栅格矩阵的行数
        image_data = dataset.ReadAsArray(0,0,im_width,im_height)
        label_data=cv2.imread(image_filepath+label_names[batch_num%len(image_names)],cv2.IMREAD_GRAYSCALE)
        train_data = []  
        train_label =  []  
        for i in range(bs):
            random_width = random.randint(0, im_width - img_w - 1)
            random_height = random.randint(0, im_height - img_h - 1)
            bands_roi=[]
            for band in image_data :
                band_roi = band[random_height: random_height + img_h, random_width: random_width + img_w]
                bands_roi.append(band_roi)
            data_roi=bands_roi
            #to_categorical(train_label, num_classes=n_label)  
            label_roi = to_categorical((label_data[random_height: random_height + img_h, random_width: random_width + img_w]).flatten(), num_classes=n_label)
            train_data.append( data_roi)  
            train_label.append(label_roi)
        #yield(np.array(train_data).shape,np.array(train_label).shape)    
        yield(np.array(train_data),np.array(train_label))
        batch_num=batch_num+1
#image_names_set=['test.tif']
#label_names_set=['test_label.png']
#for i in(generateData(8,256,256,2,image_names_set,label_names_set)):
#    print(i)

In [3]:
def SegNet():  
    #assert input_height % 32 == 0
    #assert input_width % 32 == 0
    nClasses=2
    input_height=256
    input_width=256
    img_input = Input(shape=( 4,input_height, input_width))

    # Block 1
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv1')(img_input)
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv2')(x)
    x, mask_1 = MaxPoolingWithArgmax2D(name='block1_pool')(x)

    # Block 2
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv1')(x)
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv2')(x)
    x , mask_2 = MaxPoolingWithArgmax2D(name='block2_pool')(x)

    # Block 3
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv1')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv2')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv3')(x)
    x, mask_3 = MaxPoolingWithArgmax2D(name='block3_pool')(x)

    # Block 4
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv3')(x)

    x, mask_4 = MaxPoolingWithArgmax2D(name='block4_pool')(x)

    # Block 5
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv3')(x)
    x, mask_5 = MaxPoolingWithArgmax2D(name='block5_pool')(x)

    Vgg_streamlined=Model(inputs=img_input,outputs=x)

    # 加载vgg16的预训练权重
    #Vgg_streamlined.load_weights(r"E:\Code\PycharmProjects\keras-segmentation\data\vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")

    # 解码层
    unpool_1 = MaxUnpooling2D()([x, mask_5])
    x = Conv2D(512, (3,3), padding="same")(unpool_1)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(512, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(512, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    unpool_2 = MaxUnpooling2D()([x, mask_4])
    x = Conv2D(512, (3, 3), padding="same")(unpool_2)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(512, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(256, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    unpool_3 = MaxUnpooling2D()([x, mask_3])
    y = Conv2D(256, (3, 3), padding="same")(unpool_3)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)
    y = Conv2D(256, (3, 3), padding="same")(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)
    y = Conv2D(128, (3, 3), padding="same")(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    unpool_4 = MaxUnpooling2D()([y, mask_2])
    y = Conv2D(128, (3, 3), padding="same")(unpool_4)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)
    y = Conv2D(64, (3, 3), padding="same")(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    unpool_5 = MaxUnpooling2D()([y, mask_1])
    y = Conv2D(64, (3, 3), padding="same")(unpool_5)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    y = Conv2D(nClasses, (1, 1), padding="same")(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    y = Reshape((-1, nClasses))(y)
    y = Activation("softmax")(y)

    model=Model(inputs=img_input,outputs=y)
    model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
    return model  





In [4]:
def train(): 
    EPOCHS = 1
    BS = 2
    img_w = 256  
    img_h = 256  
    model = SegNet() 
    
    modelcheck = ModelCheckpoint('..\..\Python\seg-data/model/4Bands-'+time.strftime(f'%Y-%m-%d-%a-%H-%M-%S',time.localtime(time.time()))+'.h5',monitor='val_acc',save_best_only=True,mode='max')  
    
    callable = [modelcheck]  
    
    train_numb = 1000
    valid_numb = 200
    print ("the number of train data is",train_numb,train_numb//BS)  
    print ("the number of val data is",valid_numb,valid_numb//BS)
    H = model.fit_generator(generator=generateData(BS,img_w,img_h,2,['test.tif'],['test_label.png']),
                            steps_per_epoch=train_numb//BS,
                            epochs=EPOCHS,
                            verbose=1,
                            validation_data=generateData(BS,img_w,img_h,2,['test.tif'],['test_label.png']),
                            validation_steps=valid_numb//BS,
                            callbacks=callable,
                            max_q_size=1)  

    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    N = EPOCHS
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy on SegNet Satellite Seg")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig("plot.png")



def args_parse():
    # construct the argument parse and parse the arguments
    ap = argparse.ArgumentParser()
    ap.add_argument("-a", "--augment",help="using data augment or not",
                    action="store_true", default=False)
    ap.add_argument("-m", "--model", required=True,default='model/'+time.strftime(f'%Y-%m-%d %a %H:%M:%S',time.localtime(time.time()))+'.h5'
                    ,help="path to output model")
    ap.add_argument("-p", "--plot", type=str, default="plot.png",
                    help="path to output accuracy/loss plot")
    args = vars(ap.parse_args()) 
    return args


train()

the number of train data is 1000 250
the number of val data is 200 50


c:\users\chao\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., steps_per_epoch=250, epochs=1, verbose=1, validation_data=<generator..., validation_steps=50, callbacks=[<keras.ca..., max_queue_size=1)`


Epoch 1/1
gen-Sub-Image-Data...gen-Sub-Image-Data...



MemoryError: 

In [ ]:
m = SegNet()
from keras.utils import plot_model
plot_model(m, show_shapes=True, to_file='model_segnet.png')

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
256*256*2